In [1]:
%load_ext sql
%sql sqlite:///chinook.db

## Задание 1
Вывести страну, где популярнее всего группа Iron Maiden (т.е. треков куплено больше, чем в других странах).

In [3]:
%%sql

SELECT DISTINCT c1.Country AS Country,
  (
  SELECT SUM(ii.Quantity)
    FROM invoices i
  LEFT JOIN Customers c2
    ON i.CustomerId = c2.CustomerId
  LEFT JOIN invoice_items ii
    ON ii.InvoiceId = i.InvoiceId
  LEFT JOIN tracks t
    ON ii.TrackId = t.TrackId
  LEFT JOIN albums al
    ON t.AlbumId = al.AlbumId
  LEFT JOIN artists ar
    ON al.ArtistId = ar.ArtistId
  WHERE c1.Country = c2.Country AND ar.Name LIKE "Iron Maiden"
  ) AS TracksCount
  FROM Customers c1
ORDER BY TracksCount DESC
  LIMIT 1;

 * sqlite:///chinook.db
Done.


Country,TracksCount
USA,34


In [2]:
%%sql

SELECT c.Country AS Country, SUM(ii.Quantity) AS TracksCount
  FROM Customers c
JOIN invoices i
  ON i.CustomerId = c.CustomerId
JOIN invoice_items ii
  ON ii.InvoiceId = i.InvoiceId
JOIN tracks t
  ON ii.TrackId = t.TrackId
JOIN albums al
  ON t.AlbumId = al.AlbumId
JOIN artists ar
  ON al.ArtistId = ar.ArtistId
WHERE ar.Name LIKE "Iron Maiden"
  GROUP BY c.Country
ORDER BY TracksCount DESC
  LIMIT 1;

 * sqlite:///chinook.db
Done.


Country,TracksCount
USA,34


## Задание 2
Вывести альбом, который является максимальным по продолжительности среди альбомов со стоимостью, большей средней цены за альбом *(цена альбома - сумма цен треков)*.

In [24]:
%%sql

SELECT DISTINCT a.Title AS AlbumName
  FROM Albums a
JOIN tracks t
  ON t.AlbumId = a.AlbumId
GROUP BY t.AlbumId
HAVING SUM(t.UnitPrice) >
  (
    SELECT AVG(AlbumPrice) FROM
    (
      SELECT SUM(t2.UnitPrice) AS AlbumPrice
        FROM Albums a2
      JOIN tracks t2
        ON t2.AlbumId = a2.AlbumId
      GROUP BY t2.AlbumId
    )
  )
ORDER BY SUM(t.Milliseconds) DESC
LIMIT 1;

 * sqlite:///chinook.db
Done.


AlbumName
"Lost, Season 3"


## Задание 3
Вывести общую стоимость треков, приобретённых компаниями (только клиентами из конкретных компаний), которые обслуживались сотрудниками, нанятыми после 3 августа 2002 года.

In [29]:
%%sql

SELECT SUM(t.UnitPrice) AS TracksSum
  FROM tracks t
JOIN invoice_items ii
  ON t.TrackId = ii.TrackId
JOIN invoices i
  ON ii.InvoiceId = i.InvoiceId
JOIN customers c
  ON i.CustomerId = c.CustomerId
JOIN employees e
  ON c.SupportRepId = e.EmployeeId
WHERE c.Company IS NOT NULL AND e.HireDate > "2002-08-03 00:00:00"

 * sqlite:///chinook.db
Done.


TracksSum
230.72000000000062


## Задание 4
Для каждого альбома определить все жанры песен, которые в нём есть и количество песен по каждому жанру (формат вывода: альбом, жанр, количество).
Выводить только те альбомы, где жанров больше одного.

In [110]:
%%sql

WITH AlbumGenres AS (
  SELECT a.AlbumId,g.GenreId FROM albums a
    JOIN tracks t ON a.AlbumId = t.AlbumId
    JOIN genres g ON t.GenreId = g.GenreId
  GROUP BY a.AlbumId, g.GenreId
)

SELECT a.Title AS AlbumTitle,g.Name AS GenreName, COUNT(t.TrackId) AS TrackCount FROM AlbumGenres ag
  JOIN genres g ON g.GenreId = ag.GenreId
  JOIN albums a ON a.AlbumId = ag.AlbumId
  JOIN tracks t ON a.AlbumId = t.AlbumId
  JOIN invoice_items ii ON ii.TrackId = t.TrackId
GROUP BY ag.AlbumId
HAVING COUNT(ag.GenreId) > 1
ORDER BY a.Title

 * sqlite:///chinook.db
Done.


AlbumTitle,GenreName,TrackCount
Unplugged,Blues,14
Unplugged,Latin,16
Live After Death,Metal,11
Live After Death,Heavy Metal,7
Rock In Rio [CD2],Rock,8
Rock In Rio [CD2],Metal,1
The Number of The Beast,Rock,1
The Number of The Beast,Metal,7
Greatest Hits,Rock,30
Greatest Hits,Metal,14


## Задание 5
Для каждого артиста (таблицы Artists), у которых есть песни нескольких жанров, найти жанр, по которому он заработал наибольшее количество денег, а также вывести соотношение суммы, заработанной артистом по этому жанру к общей сумме заработанных денег. Если по нескольким жанрам заработано одинаковое количество денег, максимальное для артиста, то выбирается жанр раньше по алфавиту. Формат вывода: артист, жанр, соотношение.

In [150]:
%%sql

WITH earning AS (
  SELECT ar.ArtistId AS ArtistId, gr.GenreId AS GenreId, SUM(ii.UnitPrice * ii.Quantity) AS Earning FROM artists ar
    JOIN albums al ON ar.ArtistId = al.ArtistId
    JOIN tracks tr ON al.AlbumId = tr.AlbumId
    JOIN genres gr ON gr.GenreId = tr.GenreId
    JOIN invoice_items ii ON tr.TrackId = ii.TrackId
    GROUP BY tr.GenreId,ar.ArtistId
),

max_earning AS (
  SELECT ArtistId, GenreId, MAX(DISTINCT Earning) AS MaxEarn, SUM(Earning) AS TotalEarn FROM earning
    GROUP BY ArtistId
)

SELECT ar.Name AS ArtistName, gr.Name AS GenreName, er.MaxEarn/er.TotalEarn AS Ratio FROM max_earning er
  JOIN artists ar ON ar.ArtistId = er.ArtistId
  JOIN genres gr ON gr.GenreId = er.GenreId
  JOIN earning er2 ON er2.ArtistId = er.ArtistId
GROUP BY er.ArtistId
HAVING COUNT(DISTINCT er.GenreId) > 1;


 * sqlite:///chinook.db
Done.


ArtistName,GenreName,Ratio
Amy Winehouse,R&B/Soul,0.67
Antônio Carlos Jobim,Latin,0.68
Audioslave,Rock,0.38
Battlestar Galactica,Science Fiction,0.5
Eric Clapton,Blues,0.68
Faith No More,Alternative & Punk,0.79
Foo Fighters,Rock,0.77
Gilberto Gil,Latin,0.65
Guns N' Roses,Rock,0.72
Heroes,Drama,0.85


%%sql

WITH earning AS (
  SELECT ar.ArtistId AS ArtistId, gr.GenreId AS GenreId, SUM(ii.UnitPrice * ii.Quantity) AS Earning FROM artists ar
    JOIN albums al ON ar.ArtistId = al.ArtistId
    JOIN tracks tr ON al.AlbumId = tr.AlbumId
    JOIN genres gr ON gr.GenreId = tr.GenreId
    JOIN invoice_items ii ON tr.TrackId = ii.TrackId
    GROUP BY tr.GenreId,ar.ArtistId
),

max_earning AS (
  SELECT ArtistId, GenreId, MAX(Earning) AS MaxEearn, SUM(Earning) AS TotalEearn FROM earning
    GROUP BY ArtistId
)

SELECT ar.Name AS ArtistName, m.MaxEearn/m.TotalEearn AS Ratio FROM max_earning
  JOIN artists ar ON ar.ArtistId = m.ArtistId
  JOIN genres gr ON gr.GenreId = m.GenreId
